In [3]:
from tree import get_tree_idxs
from model import KKLayout
from types import SimpleNamespace
import tensorflow as tf

In [4]:
parser = {}
parser['tree'] = 33
parser['seed']=1234
parser['opt'] = 'Adam'
parser['lr']=0.05#0.005
parser['n_epochs']=150
config = SimpleNamespace(**parser)
config.tree_idxs = get_tree_idxs(config.tree)
print(config.tree_idxs)

epoch = 0

# initialize model
if 'sess' in globals(): sess.close()
model = KKLayout(config)

sess = tf.InteractiveSession(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
C_xy0 = sess.run([model.topic_coords])


import math

print('---START TRAINING----------')
while epoch < config.n_epochs:

    max_d = 1
    feed_dict = model.get_feed_dict(max_d)
    
    _, loss_batch, global_step_log = \
    sess.run([model.opt, model.loss, tf.train.get_global_step()], feed_dict = feed_dict)
    if (epoch%1000==0):
        print(loss_batch)
    epoch += 1

{0: [1, 2, 3], 1: [11, 12, 13], 2: [21, 22, 23], 3: [31, 32, 33]}
---START TRAINING----------
17.87715


In [20]:
## Node coordinates before optimizing
C_xy0 =C_xy0[0]

## Node coordinates after optimizing
C_xy = sess.run([model.topic_coords])
C_xy = C_xy[0]



## Drawing

In [27]:
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

x_list = []
y_list = []
z_list = []
c_list = []
for topic, point in C_xy0.items():
    x_list.append(point[0][0])
    y_list.append(point[0][1])
    z_list.append(0)
    if model.node_level[topic]==0:
        c_list.append('red')
    if model.node_level[topic]==1:
        c_list.append('green')
    if model.node_level[topic]==2:
        c_list.append('blue')
    if model.node_level[topic]==3:
        c_list.append('yellow')


# c_list = ['21']+['22']*(len(list(C_xy.keys())) - 1 - np.sum(1 for e in list(C_xy.keys()) if e>19)) +['23']*np.sum(1 for e in list(C_xy.keys()) if e>19)
node_idx = model.topic_idxs
size_list = [25]*len(c_list)
df = pd.DataFrame({
    'x': x_list,
    'y': y_list,
    'z': z_list,
    'c': c_list,
    'text': node_idx,
    'size': size_list
})

import plotly.express as px
fig = px.scatter(df, x='x', y='y',
              color='c', size='size', text='text')
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
import plotly.graph_objs as go
# create edge paris
edges = [[(parent, child) for child in childs] for parent, childs in model.tree_idxs.items()]
edges = [item for sublist in edges for item in sublist]

#create new trace for line between parent->childs
x_lines = list()
y_lines = list()
z_lines = list()
for p in edges:
#     point1 = C_xy[p[0]]
#     point2 = C_xy[p[1]]
    for i in range(2):
        x_lines.append(C_xy0[p[i]][0][0])
        y_lines.append(C_xy0[p[i]][0][1])
        z_lines.append(0)
    x_lines.append(None)
    y_lines.append(None)

trace2 = go.Scatter(
    x=x_lines,
    y=y_lines,
#     z=z_lines,
    mode='lines',
)#z=z_lines,

fig.add_trace(trace2)
fig.write_image('before_kk.png')



In [25]:
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

x_list = []
y_list = []
z_list = []
c_list = []
for topic, point in C_xy.items():
    x_list.append(point[0][0])
    y_list.append(point[0][1])
    z_list.append(0)
    if model.node_level[topic]==0:
        c_list.append('red')
    if model.node_level[topic]==1:
        c_list.append('green')
    if model.node_level[topic]==2:
        c_list.append('blue')
    if model.node_level[topic]==3:
        c_list.append('yellow')


node_idx = model.topic_idxs
size_list = [25]*len(c_list)
df = pd.DataFrame({
    'x': x_list,
    'y': y_list,
    'z': z_list,
    'c': c_list,
    'text': node_idx,
    'size': size_list
})

import plotly.express as px
fig = px.scatter(df, x='x', y='y',
              color='c', size='size', text='text')
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
import plotly.graph_objs as go
# create edge paris
edges = [[(parent, child) for child in childs] for parent, childs in model.tree_idxs.items()]
edges = [item for sublist in edges for item in sublist]

#create new trace for line between parent->childs
x_lines = list()
y_lines = list()
z_lines = list()
for p in edges:
#     point1 = C_xy[p[0]]
    for i in range(2):
        x_lines.append(C_xy[p[i]][0][0])
        y_lines.append(C_xy[p[i]][0][1])
        z_lines.append(0)
    x_lines.append(None)
    y_lines.append(None)

trace2 = go.Scatter(
    x=x_lines,
    y=y_lines,
#     z=z_lines,
    mode='lines',
)#z=z_lines,

fig.add_trace(trace2)
fig.write_image('after_kk.png')

